In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


In [4]:
# Directory with 'ripe' and 'unripe' folders
images = 'D:/indhu/Datasets/Riped and Unriped Tomato Dataset/Images'
riped_images = []
unriped_images = []

In [5]:
# List the files in the directory and categorize them
for file_name in os.listdir(images):
    file_path = os.path.join(images, file_name)
    if file_name.endswith('.jpeg'):
        if 'unriped' in file_name:
            unriped_images.append(file_path)
        else:
            riped_images.append(file_path)

print(f"Ripe images: {len(riped_images)}")
print(f"Unripe images: {len(unriped_images)}")


Ripe images: 97
Unripe images: 80


In [6]:
# Define image dimensions
img_height = 150
img_width = 150

In [7]:
# Preprocess individual images
def preprocess_image(image_path, img_height, img_width):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

# Load and preprocess the images
def load_and_preprocess_images(image_paths, img_height, img_width):
    images = [preprocess_image(img_path, img_height, img_width) for img_path in image_paths]
    return np.array(images)

# Process the images
riped_images_processed = load_and_preprocess_images(riped_images, img_height, img_width)
unriped_images_processed = load_and_preprocess_images(unriped_images, img_height, img_width)

In [9]:
# Create labels for the datasets (1 for ripe, 0 for unripe)
riped_labels = np.ones(len(riped_images))
unriped_labels = np.zeros(len(unriped_images))
# Combine the datasets
X = np.concatenate((riped_images_processed, unriped_images_processed), axis=0)
y = np.concatenate((riped_labels, unriped_labels), axis=0)

In [10]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)


In [13]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 0.5506 - loss: 2.2768 - val_accuracy: 0.5556 - val_loss: 0.7310
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.6529 - loss: 0.8386 - val_accuracy: 0.8889 - val_loss: 0.2704
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.9104 - loss: 0.2673 - val_accuracy: 0.9444 - val_loss: 0.1534
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.9284 - loss: 0.1918 - val_accuracy: 0.9722 - val_loss: 0.1662
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.9652 - loss: 0.1290 - val_accuracy: 0.9722 - val_loss: 0.1601
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.9217 - loss: 0.2537 - val_accuracy: 0.9722 - val_loss: 0.1217
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.9122 - loss: 0.1675 - val_accuracy: 0.9722 - val_loss: 0.1009
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.9632 - loss: 0.1106 - val_accuracy: 0.9722 - val_loss:

In [16]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_dataset, return_dict=True).values()
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9711 - loss: 0.0825
Validation Loss: 0.9722222089767456
Validation Accuracy: 0.07958901673555374


In [17]:
# Function for predicting the class of an image
def predict_image_class(image_path, model, img_height, img_width):
    test_image = load_img(image_path, target_size=(img_height, img_width))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image / 255.0  # Normalize
    result = model.predict(test_image)
    
    if result[0][0] >= 0.5:
        return 'Ripe Tomato'
    else:
        return 'Unripe Tomato'

In [30]:
# Test prediction on a new image
test_image_path = r'D:/indhu/Datasets/Riped and Unriped Tomato Dataset/Images/riped_tomato_11.jpeg'
prediction = predict_image_class(test_image_path, model, img_height, img_width)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Ripe Tomato
